# Coleta e Consolidação de Dados

##### Coletando dados da API

Importando das bibliotecas necessárias

In [1]:
import requests
import json
import pandas as pd

Enviando requisição para a API

In [2]:
url = r'http://localhost:5000//country'
response = requests.get(url)
countries_data = response.json()

Convertando a estrutura de dados obtida para um DataFrame pandas

In [3]:
countries_df = pd.DataFrame(countries_data)

Visualizando as primeiras linhas para confirmar a estrutura

In [4]:
countries_df.head()

,AND,ARE,AFG,ATG,AIA,ALB,ARM,AGO,ATA,ARG,...,VNM,VUT,WLF,WSM,XKX,YEM,MYT,ZAF,ZMB,ZWE
name,Andorra,United Arab Emirates,Afghanistan,Antigua and Barbuda,Anguilla,Albania,Armenia,Angola,Antarctica,Argentina,...,Vietnam,Vanuatu,Wallis and Futuna,Samoa,Kosovo,Yemen,Mayotte,South Africa,Zambia,Zimbabwe
currency,EUR,AED,AFN,XCD,XCD,ALL,AMD,AOA,,ARS,...,VND,VUV,XPF,WST,EUR,YER,EUR,ZAR,ZMW,ZWL
population,77006,9630959,37172386,96286,13254,2866376,2951776,30809762,0,44494502,...,95540395,292680,16025,196130,1845300,28498687,279471,57779622,17351822,14439018
capital,Andorra la Vella,Abu Dhabi,Kabul,St John's,The Valley,Tirana,Yerevan,Luanda,,Buenos Aires,...,Hanoi,Port Vila,Mata-Utu,Apia,Pristina,Sanaa,Mamoudzou,Pretoria,Lusaka,Harare
continent,Europe,Asia,Asia,North America,North America,Europe,Asia,Africa,Antarctica,South America,...,Asia,Oceania,Oceania,Oceania,Europe,Asia,Africa,Africa,Africa,Africa


#### Carregando Dados de PIB (GDP) do Arquivo CSV

Utilizando o pandas para carregar o CSV que contém os dados de PIB

In [8]:
gdp_df = pd.read_csv('../data/gdp_data.csv')

Visualizando as primeiras linhas

In [9]:
print(gdp_df.head())

                  Country Name Country Code     Indicator Name  \
0                        Aruba          ABW  GDP (current US$)   
1  Africa Eastern and Southern          AFE  GDP (current US$)   
2                  Afghanistan          AFG  GDP (current US$)   
3   Africa Western and Central          AFW  GDP (current US$)   
4                       Angola          AGO  GDP (current US$)   

   Indicator Code          1960          1961          1962          1963  \
0  NY.GDP.MKTP.CD           NaN           NaN           NaN           NaN   
1  NY.GDP.MKTP.CD  2.129081e+10  2.180870e+10  2.370727e+10  2.821034e+10   
2  NY.GDP.MKTP.CD  5.377778e+08  5.488889e+08  5.466667e+08  7.511112e+08   
3  NY.GDP.MKTP.CD  1.040414e+10  1.112789e+10  1.194319e+10  1.267633e+10   
4  NY.GDP.MKTP.CD           NaN           NaN           NaN           NaN   

           1964          1965  ...          2013          2014          2015  \
0           NaN           NaN  ...  2.727933e+09  2.791061e+

#### Consolidando os dados de Países e PIB

Verificando a coluna comum entre os dados

In [21]:
print("Colunas no DataFrame de países:")
print(countries_df.columns)

print("\nColunas no DataFrame de PIB:")
print(gdp_df.columns)

Colunas no DataFrame de países:
Index(['AND', 'ARE', 'AFG', 'ATG', 'AIA', 'ALB', 'ARM', 'AGO', 'ATA', 'ARG',
       ...
       'VNM', 'VUT', 'WLF', 'WSM', 'XKX', 'YEM', 'MYT', 'ZAF', 'ZMB', 'ZWE'],
      dtype='object', length=250)

Colunas no DataFrame de PIB:
Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       'Unnamed: 66'],
      dtype='object')


In [13]:
print(gdp_df['Country Code'].head())

0    ABW
1    AFE
2    AFG
3    AFW
4    AGO
Name: Country Code, dtype: object


Com base nas estruturas encontradas, parece que o código do país (Country Code no CSV e o próprio identificador no JSON) é o atributo comum entre as duas fontes de dados. Portanto, esse atributo será usado para consolidar os dados.

Transformando os dados da API

In [28]:
api_df = countries_df.T
api_df.reset_index(inplace=True)
api_df.columns = ["code", "name", "currency", "population","capital", "continent", "area"]

Verificando o resultado da transposição

In [29]:
print(api_df.head())

  code                  name currency population           capital  \
0  AND               Andorra      EUR      77006  Andorra la Vella   
1  ARE  United Arab Emirates      AED    9630959         Abu Dhabi   
2  AFG           Afghanistan      AFN   37172386             Kabul   
3  ATG   Antigua and Barbuda      XCD      96286         St John's   
4  AIA              Anguilla      XCD      13254        The Valley   

       continent      area  
0         Europe     468.0  
1           Asia   82880.0  
2           Asia  647500.0  
3  North America     443.0  
4  North America     102.0  


Organizando e transformando os dados do CSV

In [30]:
selected_columns = ['Country Code'] + [str(ano) for ano in range(1960, 2022)]
csv_df = gdp_df[selected_columns]
csv_df.rename(columns={'Country Code': 'code'}, inplace=True)

C:\Users\Marcel Albuquerque\AppData\Local\Temp\ipykernel_48452\3764212746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_df.rename(columns={'Country Code': 'code'}, inplace=True)


Verificando o resultado

In [31]:
print(csv_df.head())

  code          1960          1961          1962          1963          1964  \
0  ABW           NaN           NaN           NaN           NaN           NaN   
1  AFE  2.129081e+10  2.180870e+10  2.370727e+10  2.821034e+10  2.611906e+10   
2  AFG  5.377778e+08  5.488889e+08  5.466667e+08  7.511112e+08  8.000000e+08   
3  AFW  1.040414e+10  1.112789e+10  1.194319e+10  1.267633e+10  1.383837e+10   
4  AGO           NaN           NaN           NaN           NaN           NaN   

           1965          1966          1967          1968  ...          2012  \
0           NaN           NaN           NaN           NaN  ...  2.615084e+09   
1  2.968249e+10  3.223946e+10  3.351491e+10  3.652187e+10  ...  9.725734e+11   
2  1.006667e+09  1.400000e+09  1.673333e+09  1.373333e+09  ...  2.020357e+10   
3  1.486223e+10  1.583259e+10  1.442604e+10  1.488035e+10  ...  7.360399e+11   
4           NaN           NaN           NaN           NaN  ...  1.249982e+11   

           2013          2014         

Junção dos dados

In [33]:
consolidated_data = pd.concat([api_df.set_index('code'), csv_df.set_index('code')], axis=1).reset_index()

Verificando o DataFrame consolidado

In [34]:
print(consolidated_data.head())

  code                  name currency population           capital  \
0  AND               Andorra      EUR      77006  Andorra la Vella   
1  ARE  United Arab Emirates      AED    9630959         Abu Dhabi   
2  AFG           Afghanistan      AFN   37172386             Kabul   
3  ATG   Antigua and Barbuda      XCD      96286         St John's   
4  AIA              Anguilla      XCD      13254        The Valley   

       continent      area          1960          1961          1962  ...  \
0         Europe     468.0           NaN           NaN           NaN  ...   
1           Asia   82880.0           NaN           NaN           NaN  ...   
2           Asia  647500.0  5.377778e+08  5.488889e+08  5.466667e+08  ...   
3  North America     443.0           NaN           NaN           NaN  ...   
4  North America     102.0           NaN           NaN           NaN  ...   

           2012          2013          2014          2015          2016  \
0  3.188809e+09  3.193704e+09  3.271808e+